<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

2024-11-22 06:36:56,298 INFO: Failed to extract font properties from /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf: In FT2Font: Can not load face (unknown file format; error code 0x2)


2024-11-22 06:36:56,343 INFO: generated new fontManager


## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "hnoLoEhnnXr0NIlt.hIAb7iCp86nnF4QNTjpsgJrURRwFTMG80Pe0JrMQjwa31eQCnTEr0cVjtyb8K9Tn"

project = hopsworks.login(project="id2223_labs_24")
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1161371


Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "stockholm", "street": "st eriksgatan 83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

air_quality_lagged_fg = fs.get_feature_group(
    name='air_quality_lagged',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [4]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,7.0,sweden,stockholm,st eriksgatan 83,2024-11-22,https://api.waqi.info/feed/@10523


In [5]:
# Convert float32 columns to float64 (double)
aq_today_df = aq_today_df.astype({
    'pm25': 'float64'
})
aq_today_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 176.0+ bytes


In [6]:
data_df = air_quality_fg.read()
sorted_df = data_df.sort_values('date')
last_three_rows = sorted_df.tail(4)
last_three_rows

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.15s) 


,date,pm25,country,city,street,url
2181,2024-11-18 00:00:00+00:00,17.0,sweden,stockholm,st eriksgatan 83,https://api.waqi.info/feed/@10523
2182,2024-11-19 00:00:00+00:00,19.0,sweden,stockholm,st eriksgatan 83,https://api.waqi.info/feed/@10523
2183,2024-11-20 00:00:00+00:00,19.0,sweden,stockholm,st eriksgatan 83,https://api.waqi.info/feed/@10523
2184,2024-11-21 00:00:00+00:00,19.0,sweden,stockholm,st eriksgatan 83,https://api.waqi.info/feed/@10523


In [7]:
pm25_lag_1 = last_three_rows.iloc[-2]['pm25']  
pm25_lag_2 = last_three_rows.iloc[-3]['pm25']  
pm25_lag_3 = last_three_rows.iloc[-4]['pm25']  

In [8]:
aq_today_df['pm25lag1'] = pm25_lag_1
aq_today_df['pm25lag2'] = pm25_lag_2
aq_today_df['pm25lag3'] = pm25_lag_3
aq_today_df = aq_today_df.drop(columns=['url', 'country'])


In [9]:
# Insert new data with lagged features into the lagged feature group
air_quality_lagged_fg.insert(aq_today_df[['city', 'street', 'date', 'pm25', 'pm25lag1', 'pm25lag2', 'pm25lag3']])


2024-11-22 06:37:02,522 INFO: 	1 expectation(s) included in expectation_suite.


Validation succeeded.


Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161371/fs/1152074/fg/1347995


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_lagged_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161371/jobs/named/air_quality_lagged_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7ff5861db040>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 671796
         }
       },
       "result": {
         "observed_value": 7.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-22T06:37:02.000522Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [10]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-22,-4.95,0.0,14.058450,256.675446,stockholm
1,2024-11-23,-4.15,0.0,11.885453,271.735657,stockholm
2,2024-11-24,3.50,0.0,17.533146,160.820892,stockholm
3,2024-11-25,9.20,0.1,22.596991,202.479477,stockholm
4,2024-11-26,6.15,0.0,13.891522,211.218338,stockholm
5,2024-11-27,4.30,0.0,11.874544,14.036275,stockholm
6,2024-11-28,5.00,0.1,11.440978,12.724360,stockholm
7,2024-11-29,3.15,0.0,6.489992,250.559875,stockholm
8,2024-11-30,4.75,0.0,19.008547,245.376450,stockholm
9,2024-12-01,5.90,0.0,18.514643,243.435013,stockholm


In [11]:
# Check the result
daily_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [12]:
# Insert new data
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)

aq_today_df = aq_today_df.astype({
    'pm25': 'float64'
})

air_quality_fg.insert(aq_today_df)

2024-11-22 06:37:10,732 INFO: 	1 expectation(s) included in expectation_suite.


Validation succeeded.


Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161371/fs/1152074/fg/1351080


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161371/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7ff58555f100>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 674943
         }
       },
       "result": {
         "observed_value": 7.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-22T06:37:10.000732Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [13]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-22 06:37:17,465 INFO: 	2 expectation(s) included in expectation_suite.


Validation succeeded.


Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1161371/fs/1152074/fg/1351081


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1161371/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7ff5861db7c0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 674945
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-22T06:37:17.000465Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betw

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
